In [3]:
import mdtraj as md
import MDAnalysis as mda
import numpy as np
from scipy.stats import ttest_ind_from_stats

from tqdm import tqdm
from pathlib import Path
import os
from natsort import natsorted
import pandas as pd
from addict import Dict as Adict

import pyemma as pm
import deeptime as dt
import deeptime.markov.msm as msm
import deeptime.markov.hmm as hmm

from deeptime.plots import plot_implied_timescales, plot_energy2d, plot_contour2d_from_xyz
from deeptime.markov.sample import *
from deeptime.markov import TransitionCountEstimator
from deeptime.util import energy2d

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns

from funcs_build_msm import _tica, _kmeans, get_data
from funcs_validate_msm import *
from funcs_sample import *
from funcs_plotting import *
from funcs_characterise import *
from paths import *

In [4]:
study_name = 'n_clusters'
hp_id = 1
savedir = Path(f'data_egfr/msm/validation/{study_name}_hp{hp_id}_validate')

n_states = 6

### Compute the RMSDs within each sampled states

In [ ]:
for state_i in range(n_states):
    rmsd_mean, rmsd_std = cal_within_state_rmsd(md.load(savedir/f'MS_{state_i+1}.pdb'))
    save_example_with_property(savedir/f'MS_{state_i+1}.pdb', rmsd_mean, savedir/f'MS_{state_i+1}_rmsd_mean.pdb')

### Compute the different scores between every two pairs of states

In [5]:
samples = [md.load(savedir/f'MS_{state_i+1}.pdb') for state_i in range(n_states)]
scores = []

In [38]:
res_scores = []

for i in range(n_states):
    print(i)
    res_scores.append([])
    rmsd_mean_i, rmsd_std_i = cal_within_state_rmsd(samples[i])
    for j in range(n_states):
        if j > i:
            print(j)
            rmsd_mean_j, rmsd_std_j = cal_within_state_rmsd(samples[j])
            rmsd_mean_ij, rmsd_std_ij = cal_between_states_rmsd(samples[i], samples[j])
            res_scores[i].append(cal_between_states_diff(rmsd_mean_ij, rmsd_std_i, rmsd_std_j))

0
1
2
3
4
5
1
2
3
4
5
2
3
4
5
3
4
5
4
5
5


In [42]:
res_scores[0][1]

array([0.06081811, 0.14064138, 0.27105602, 0.29951664, 0.30701242,
       0.23471415, 0.19009224, 0.20416561, 0.26885578, 0.22991652,
       0.18764016, 0.1883268 , 0.16138029, 0.15154013, 0.12942757,
       0.11560587, 0.09755897, 0.06928828, 0.06438298, 0.06837926,
       0.09348713, 0.10200429, 0.1288904 , 0.16660005, 0.21749881,
       0.20197073, 0.22522704, 0.22839701, 0.20949871, 0.22172158,
       0.13831426, 0.06693547, 0.03269809, 0.04769845, 0.06263318,
       0.09022541, 0.12733271, 0.18807006, 0.21523726, 0.2310803 ,
       0.26563426, 0.30046725, 0.30188168, 0.22015786, 0.13987225,
       0.05583059, 0.02851513, 0.00982116, 0.        , 0.01654207,
       0.04772185, 0.12201295, 0.13445244, 0.11246471, 0.13121728,
       0.13593265, 0.15435705, 0.16867773, 0.13988637, 0.20993522,
       0.34178979, 0.24086879, 0.22505938, 0.3076943 , 0.26653167,
       0.16889534, 0.26373586, 0.38844449, 0.44021324, 0.35844743,
       0.50123678, 0.48906884, 0.49325903, 0.4432007 , 0.43834

In [46]:
for i in range(n_states):
    for j in range(n_states-i-1):
            save_example_with_property(savedir/f'MS_{i+1}.pdb', res_scores[i][j], savedir/f'MS_{i+1}_{j+i+2}_score.pdb')

/home/rzhu/Loc/miniconda3/envs/kinase/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/rzhu/Loc/miniconda3/envs/kinase/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


### Inspect samples of dfg states

In [3]:
dfg_in_mean, dfg_in_std = cal_within_state_rmsd(dfg_in_samples)
dfg_out_mean, dfg_out_std = cal_within_state_rmsd(dfg_out_samples)
in_out_mean, in_out_std = cal_between_states_rmsd(dfg_in_samples, dfg_out_samples)
score = cal_between_states_diff(in_out_mean, dfg_in_std, dfg_out_std)

In [4]:
save_example_with_property("./data_egfr/msm/dunbrack_studies/lag100_random_trials/61/MS_0.pdb", score, 'rmsd_diff_dfg-in-out.pdb')

/home/rzhu/Loc/miniconda3/envs/kinase/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/rzhu/Loc/miniconda3/envs/kinase/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


In [ ]:
save_example_with_property("./data_egfr/msm/dunbrack_studies/lag100_random_trials/61/MS_0.pdb", score, 'rmsd_diff_dfg-in-out.pdb')

In [ ]:
save_example_with_property("./data_egfr/msm/dunbrack_studies/lag100_random_trials/61/MS_0.pdb", score, 'rmsd_diff_dfg-in-out.pdb')